## Machine Unlearning (MU) for Erase diff algorithm. 

 #### **1: Environment Setup**
 
In this section, we set up our Python environment and install the necessary packages. For reproducibility, it’s best to use a virtual environment.


Instructions:

* Create and activate your virtual environment.

* Install our package `unlearn_diff`.


**Prerequisities**

Ensure conda is installed on your system. You can install Miniconda or Anaconda:

* Miniconda (recommended): https://docs.conda.io/en/latest/miniconda.html

* Anaconda: https://www.anaconda.com/products/distribution

After installing conda, ensure it is available in your PATH by running. You may require to restart the terminal session:

In [ ]:
#create venv if you are using python script.

!python -m venv unlearning-env

After creating your virtual environment, if you're working in a notebook, select the kernel associated with that environment. If you're using a terminal instead, activate the environment with:

`source unlearning-env/bin/activate`

In [ ]:
# Install our packaging unlearn_diff

!pip install unlearn_diff

#### 2. Create unlearn_diff conda environment  and activate

Use this command to create environment:

`create_env`



In [ ]:
!create_env

Activate conda environment:

`conda activate <env_name>`

#### **3. Downloading the Dataset**

After you install the package, you can use the following commands to download.

1. i2p:

* Sample: 

```bash 
     download_data sample i2p
```

* Full: 
```bash 
     download_data full i2p
```

2. quick_canvas:

* Sample: 

```bash 
     download_data sample quick_canvas
```

* Full: 

```bash 
     download_data full quick_canvas
```

In [ ]:
# downloading sample i2p dataset

!download_data sample i2p

In [ ]:
#downloading sample quick_canvas dataset:

!download_data sample quick_canvas

Once you have downloaded datasets, verify the Downloaded Files.

* ls data/i2p-dataset/sample/

* ls -lh ./data/quick-canvas-dataset/sample/

#### **4. Downloading models**

* compvis: 

```bash 
     download_model compvis
```

* diffuser: 

```bash
     download_model diffuser
```

In [ ]:
# download compvis model

#! download_model compvis 

In [ ]:
# download diffuser model

#! download_model diffuser

#### 5. **MU**

**Run Train**

The default configuration for training is provided by erase_diff_train_mu. You can run the training with the default settings as follows:


```python
from mu.algorithms.erase_diff.algorithm import EraseDiffAlgorithm
from mu.algorithms.erase_diff.configs import erase_diff_train_mu

algorithm = EraseDiffAlgorithm(
    erase_diff_train_mu
)
algorithm.run()
```

<br> <br>

**Overriding the Default Configuration**

If you need to override the existing configuration settings, you can specify your custom parameters (such as ckpt_path and raw_dataset_dir) directly when initializing the algorithm. For example:

```python
from mu.algorithms.erase_diff.algorithm import EraseDiffAlgorithm
from mu.algorithms.erase_diff.configs import erase_diff_train_mu

algorithm = EraseDiffAlgorithm(
    erase_diff_train_mu,
    use_sample = True
)
algorithm.run()
```

<span style="color: red;"><br>Note: When fine-tuning the model, if you want to use a sample dataset, set use_sample=True (default).Otherwise, set use_sample=False to use the full dataset.<br></span>

In [ ]:
# machine unlearning with quick canvas dataset


from mu.algorithms.erase_diff.algorithm import EraseDiffAlgorithm
from mu.algorithms.erase_diff.configs import erase_diff_train_mu

algorithm = EraseDiffAlgorithm(
    erase_diff_train_mu,
    ckpt_path="/home/ubuntu/Projects/UnlearnCanvas/UnlearnCanvas/machine_unlearning/models/compvis/style50/compvis.ckpt",
    raw_dataset_dir="data/quick-canvas-dataset/sample",
    use_sample = True, #uses sample dataset
    template_name = "Abstractionism",
    dataset_type = "unlearncanvas",
    devices = "0"
)
algorithm.run()


In [ ]:
# Machine unlearning with i2p dataset


from mu.algorithms.erase_diff.algorithm import EraseDiffAlgorithm
from mu.algorithms.erase_diff.configs import erase_diff_train_i2p

algorithm = EraseDiffAlgorithm(
    erase_diff_train_i2p,
    ckpt_path="/home/ubuntu/Projects/UnlearnCanvas/UnlearnCanvas/machine_unlearning/models/compvis/style50/compvis.ckpt",
    raw_dataset_dir="data/i2p-dataset/sample",
    num_samples = 1,
    dataset_type = "i2p",
    template = "i2p",
    template_name = "self-harm",
    use_sample = True #uses sample dataset
    
)
algorithm.run()

#### Evaluation of MU

The evaluation framework is used to assess the performance of models after applying machine unlearning.

config descriptions:

* erase_diff_evaluation_config : default evaluation config for erase_dif
* ckpt_path: finetuned model path for erase_diff algorithm.
* classifier_ckpt_path: Path to classifier model
* refrence_dir: original dataset dir.

In [ ]:
from mu.algorithms.erase_diff import EraseDiffEvaluator
from mu.algorithms.erase_diff.configs import (
    erase_diff_evaluation_config
)

evaluator = EraseDiffEvaluator(
    erase_diff_evaluation_config,
    ckpt_path="outputs/erase_diff/finetuned_models/erase_diff_Abstractionism_model.pth", #finetuned model path 
    classifier_ckpt_path = "/home/ubuntu/Projects/models/classifier_ckpt_path/style50_cls.pth",
    reference_dir= "data/quick-canvas-dataset/sample/"
)
evaluator.run()

<span style="color:red;">**Output for evalaution will be saved to the given directory `Outputs/eval_results/mu_results/erase_diff/`** </span>

#### **Run Attacks**

Before running attacks, download dataset for attack. Run the following command in terminal.

`generate_attack_dataset --prompts_path data/prompts/prompts.csv --concept i2p_nude --save_path outputs/dataset --num_samples 1`

Here, prompts_path is the path of csv containing prompt, concept is the name of the concept for organizing output file, save_path is the directory where generated images and metadata will be saved, num_samples is the number of images to generate per prompt.

1. **Hard Prompt Attack - compvis**

After performing unlearning, make sure to use the same fine-tuned model checkpoint (generated during unlearning) for performing the attack.
Use the following code if you wish to run the hard prompt attack using the CompVis model directly (without converting it into Diffusers format):

In [ ]:
from mu_attack.configs.nudity import no_attack_esd_nudity_classifier_compvis_config
from mu_attack.execs.attack import MUAttack
from mu.algorithms.erase_diff.configs import erase_diff_train_mu

def run_attack_for_nudity():

    overridable_params = {
    "task.compvis_ckpt_path" :"outputs/erase_diff/finetuned_models/erase_diff_Abstractionism_model.pth",
    "task.dataset_path" : "/home/ubuntu/Projects/Palistha/unlearn_diff_attack/outputs/dataset/i2p_nude",
   "task.compvis_config_path" : erase_diff_train_mu.model_config_path ,
    "attacker.no_attack.dataset_path" : "/home/ubuntu/Projects/Palistha/unlearn_diff_attack/outputs/dataset/i2p_nude",
    "logger.json.root" : "results/no_attack_esd_nudity_P4D_erase_diff"
    }

    MUAttack(
        config=no_attack_esd_nudity_classifier_compvis_config,
        **overridable_params
    )

if __name__ == "__main__":
    run_attack_for_nudity()


**Hard Prompt Attack – CompVis to Diffusers Conversion**

If you want to convert the CompVis model into the Diffusers format before running the attack, use the following code. Note: For the conversion to take place, set task.save_diffuser to True and to use the converted model task.sld should be set to None.

Note:
1.  Enable conversion: When True, the CompVis model is converted to the Diffuser format during the attack process, allowing integration with the Diffusers library for further processing.

2. SLD conversion parameters by setting to None. This ensures that no additional SLD modifications interfere with the conversion of the CompVis model to the Diffuser format.

In [ ]:
from mu_attack.configs.nudity import hard_prompt_esd_nudity_P4D_compvis_config
from mu_attack.execs.attack import MUAttack
from mu.algorithms.erase_diff.configs import erase_diff_train_mu

def run_attack_for_nudity():

    overridable_params = {
        "task.compvis_ckpt_path":"outputs/erase_diff/finetuned_models/erase_diff_Abstractionism_model.pth",
        "task.compvis_config_path": erase_diff_train_mu.model_config_path,
        "task.dataset_path":"/home/ubuntu/Projects/Palistha/msu_unlearningalgorithm/outputs/dataset/i2p_nude",
        "logger.json.root":"results/hard_prompt_esd_nudity_P4D_scissorhands",
        "attacker.no_attack.dataset_path" : "/home/ubuntu/Projects/Palistha/unlearn_diff_attack/outputs/dataset/i2p_nude",
        "task.save_diffuser": True, # This flag triggers conversion of compvis model to diffuser while performinig attack
        "task.sld": None, # Set sld to None for conversion of compvis model to diffuser while performinig attack
        "task.model_name": "SD-v1-4"
    }

    MUAttack(
        config=hard_prompt_esd_nudity_P4D_compvis_config,
        **overridable_params
    )

if __name__ == "__main__":
    run_attack_for_nudity()

**No Attack - compvis**

Use the following code if you wish to run the No attack using the CompVis model directly (without converting it into Diffusers format):

In [ ]:
from mu_attack.configs.nudity import no_attack_esd_nudity_classifier_compvis_config
from mu_attack.execs.attack import MUAttack
from mu.algorithms.erase_diff.configs import erase_diff_train_mu

def run_attack_for_nudity():

    overridable_params = {
    "task.compvis_ckpt_path" :"outputs/erase_diff/finetuned_models/erase_diff_Abstractionism_model.pth",
   "task.compvis_config_path" : erase_diff_train_mu.model_config_path ,
    "task.dataset_path" : "/home/ubuntu/Projects/Palistha/msu_unlearningalgorithm/outputs/dataset/i2p_nude",
    "logger.json.root" : "results/no_attack_esd_nudity_P4D_erase_diff"
    }

    MUAttack(
        config=no_attack_esd_nudity_classifier_compvis_config,
        **overridable_params
    )

if __name__ == "__main__":
    run_attack_for_nudity()


**No Attack – CompVis to Diffusers Conversion**

If you want to convert the CompVis model into the Diffusers format before running the attack, use the following code. Note: For the conversion to take place, set task.save_diffuser to True and to use the converted model task.sld should be set to None.


Note:
1.  Enable conversion: When True, the CompVis model is converted to the Diffuser format during the attack process, allowing integration with the Diffusers library for further processing.

2. SLD conversion parameters by setting to None. This ensures that no additional SLD modifications interfere with the conversion of the CompVis model to the Diffuser format.

In [ ]:

from mu_attack.configs.nudity import no_attack_esd_nudity_classifier_compvis_config
from mu_attack.execs.attack import MUAttack
from mu.algorithms.erase_diff.configs import erase_diff_train_mu

def run_attack_for_nudity():

    overridable_params = {
    "task.compvis_ckpt_path" :"outputs/erase_diff/finetuned_models/erase_diff_Abstractionism_model.pth",
    "task.compvis_config_path" : erase_diff_train_mu.model_config_path ,
    "task.dataset_path" : "/home/ubuntu/Projects/Palistha/msu_unlearningalgorithm/outputs/dataset/i2p_nude",
    "logger.json.root" : "results/hard_prompt_esd_nudity_P4D_erase_diff_compvis_to_diffuser",
    "task.save_diffuser": True, #This flag triggers conversion of compvis model to diffuser while performinig attack
    "task.sld": None, #  sld should be set to None while conversion of compvis model to diffuser while performinig attack
    "task.model_name": "SD-v1-4"
    }

    MUAttack(
        config=no_attack_esd_nudity_classifier_compvis_config,
        **overridable_params
    )

if __name__ == "__main__":
    run_attack_for_nudity()



#### Evaluation of MU_attack

Calculate the performance of mu_attack using logs and images generated during attack.

In [ ]:
from mu_attack.configs.evaluation import attack_evaluation_config
from mu_attack.execs.evaluator import MuAttackEvaluator

def main():
    config = attack_evaluation_config
    config.asr.root = "results/hard_prompt_esd_nudity_P4D_abstractionism_diffuser/P4d"
    config.asr.root_no_attack = "results/no_attack_esd_nudity_P4D_erase_diff/NoAttackEsdNudity"
    config.clip.devices = "0"
    config.clip.image_path = "results/hard_prompt_esd_nudity_P4D_abstractionism_diffuser/P4d/images"
    config.clip.log_path = "results/hard_prompt_esd_nudity_P4D_abstractionism_diffuser/P4d/log.json"
    config.fid.ref_batch_path = "results/no_attack_esd_nudity_P4D_erase_diff/NoAttackEsdNudity/images"  #to calculate fid score, the genrated image path and ref image path should have ssame number of images.
    config.fid.sample_batch_path = "/home/ubuntu/Projects/Palistha/msu_unlearningalgorithm/data/i2p/nude"

    # Common output path
    config.output_path = "results/evaluation/results.json"

    evaluator = MuAttackEvaluator(config)

    # Run the evaluation (this will run ASR, CLIP, and FID evaluators)
    results = evaluator.run()

    print("Evaluation Results:",results)

if __name__ == "__main__":
    main()

### Mu Defense (AdvUnlean)

After performing unlearning and attack, we need to perform adversarial unlearning by integrating a soft prompt attack into the training loop. use the following code snippet for advunlearn.


Note: import the model_config_path from the relevant algorithm's configuration module in the mu package



In [ ]:
from mu_defense.algorithms.adv_unlearn.algorithm import AdvUnlearnAlgorithm
from mu_defense.algorithms.adv_unlearn.configs import adv_unlearn_config
from mu.algorithms.erase_diff.configs import erase_diff_train_mu

def mu_defense():

    mu_defense = AdvUnlearnAlgorithm(
        config=adv_unlearn_config,
        compvis_ckpt_path = "outputs/erase_diff/finetuned_models/erase_diff_Abstractionism_model.pth", #path to the finetuned model
        attack_step = 2,
        backend = "compvis",
        attack_method = "fast_at",
        train_method = "text_encoder_full",  #training method. check for docs for available train_method 
        # train_method = "noxattn", #training method. check for docs for available train_method 
        warmup_iter = 1,
        iterations = 2,
        model_config_path = erase_diff_train_mu.model_config_path  # use same model_config_path for the model you are using

    )
    mu_defense.run()

if __name__ == "__main__":
    mu_defense()

### Evaluation for mu_defense

Before proceeding with evaluation, you must generate images using the output from mu_defense.

To generate images use the following code snippet:


Description of params used:

* config: default train config for image generation.
* target_ckpt: Model ckpt after running mu_defense (AdvUnleran).
* model_config_path: model config path for the model being used for defense.
* save_path: output dir to save generated images.
* prompts_path: path to the csv with prompts.
* num_samples: number of samples to be generated for a prompt.
* folder_suffix: suffix for folder name for save path.
* devices: devices to be used.  

In [ ]:
from mu_defense.algorithms.adv_unlearn.configs import example_image_generator_config
from mu_defense.algorithms.adv_unlearn import ImageGenerator
from mu.algorithms.erase_diff.configs import erase_diff_train_mu

def generate_image():
    generate_image = ImageGenerator(
        config = example_image_generator_config,
        target_ckpt = "outputs/results_with_retaining/nudity/coco_object/pgd/AttackLr_0.001/text_encoder_full/all/prefix_k/AdvUnlearn-nudity-method_text_encoder_full_all-Attack_pgd-Retain_coco_object_iter_1.0-lr_1e-05-AttackLr_0.001-prefix_k_adv_num_1-word_embd-attack_init_latest-attack_step_30-adv_update_1-warmup_iter_200/models/TextEncoder-text_encoder_full-epoch_1.pt",
        model_config_path = erase_diff_train_mu.model_config_path,
        save_path = "outputs/adv_unlearn/models",
        prompts_path = "data/prompts/sample_prompts.csv",
        num_samples = 1,
        folder_suffix = "imagenette",
        devices = "0"

    )
    generate_image.generate_images()

if __name__ == "__main__":
    generate_image()

Once the images are generated using the finetuned model from defense, you need to evaulate it's performance: 

Note: Before performing evalaution:
1. Download coco 10k dataset from this link : https://drive.google.com/file/d/1Qgm3nNhp6ykamszN_ZvofvuzjryTsPHB/view 
2. If you want to evaluate just clip score or FID score then mention job as clip or fid accordingly. If you mention nothing then it will calculate both.

In [ ]:

from mu_defense.algorithms.adv_unlearn import MUDefenseEvaluator
from mu_defense.algorithms.adv_unlearn.configs import mu_defense_evaluation_config

def mu_defense_evaluator():
    evaluator = MUDefenseEvaluator(
        config = mu_defense_evaluation_config,
        gen_imgs_path = "outputs/adv_unlearn/models_visualizations_imagenette/SD-v1-4/",
        coco_imgs_path = "coco_dataset/sample/",
        output_path = "outputs/adv_unlearn/evaluation/",
        devices = "0"
        # job = ""  # "clip"  or "fid"
    )
    evaluator.run()


if __name__ == "__main__":
    mu_defense_evaluator()